This is a protopype for artificial neural netwok (I believe it is called a feed-forward ann, or something).

In the data.txt, there are training data for the OR gate, but in principle you can use whatever you want.
The limitaion of this particular implementation is that it only has one output. This is because I 
wrote it in just a few days (I had no idea about anns before starting writing), since I had to also 
finish my PhD. But I should be able to easily generalize it.

The code is ugly, but works. It still needs some good convergence criteria.
I hope to update it and make it better and clearer in the future.

In [1]:
import numpy as np

In [2]:
# sigmoid function
def nonlin(x,deriv=False):
    if(deriv==True):
        return nonlin(x)*(1-nonlin(x))
    return 1/(1+np.exp(-x))

def rnonlin(x,deriv=False):
    if(deriv==True):
        return nonlin(x,deriv)
    return np.round(nonlin(x,deriv),0)

def lin(x,deriv=False):
    if(deriv==True):
        return 1*np.power(x,0)
    return np.power(x,1)

def polyn(n,x,deriv=False):
    if(deriv==True):
        return n*np.power(x,float(n-1.))
    return np.power(x,float(n))

def relu(x,deriv=False):
    relu_temp=[]
    if deriv==False:
        for i in x:
            if i<=0:
                relu_temp.append(0)
            else:
                relu_temp.append(i)
        return relu_temp
    if deriv:
            if x<=0:
                return 0
            else:
                return 1
            
def relu_polyn(n,x,deriv=False):
    relu_temp=[]
    if deriv==False:
        for i in x:
            if i<=0:
                relu_temp.append(0)
            else:
                relu_temp.append(polyn(n,i,deriv))
        return relu_temp
    if deriv:
            if x<=0:
                return 0
            else:
                return polyn(n,x,deriv)       


    
def sqerr(prediction,target,deriv=False):
    if(deriv==True):
        return prediction-target
    return 1/2.*np.power(prediction-target,2)



#output_func is usually a linear function (because it can produce any real number), but play around and observe  
def output_func(x,deriv=False):
    return lin(x,deriv) #this gives 0 error!
    #return np.round(nonlin(x,deriv),2) #it's good!
    #return rnonlin(x,deriv)#this gives 0 error!
    #return nonlin(x,deriv)

def hidden_func(x,deriv=False): #define the function of the hidden layers
    return nonlin(x,deriv)
    #return lin(x,deriv)
    #return polyn(1,x,deriv)

def error_func(prediction,target,deriv=False):
    return sqerr(prediction,target,deriv)

#--------------------------------------------------------------------------------------------------
def X_calculation(Signals,Inputs,Weights,Layers ):
    for l in range(Layers-2): #range(L-2) because I'm going to fill the last xl and last signal outside the loop!
        Signals[l]=np.array(Weights[l]).T.dot(np.array(Inputs[l]))
        Inputs[l+1]=np.append(hidden_func(Signals[l]),1) #update xl and add the bias!
    #the last signal: 
    Signals[Layers-2]=np.array(Weights[Layers-2]).T.dot(np.array(Inputs[Layers-2]))
    Inputs[Layers-1]=output_func(Signals[Layers-2])


def W_update(Inputs,Deltas,Layers,Layer_dim,L_Rate,W):
    for l in range(0,Layers-1):    
            for i in range(Layer_dim[l]+1):
                for j in range(0,Layer_dim[l+1]):
                    W[l][i][j]+=-L_Rate*Inputs[l][i]*Deltas[l][j] 
                    
def BackProp(Signals,Weights,Error,Deltas,Layers,Layer_dim):
    Deltas[L-2][0]=Error*output_func(Signals[Layers-2][0],True)
    for l in reversed(range(Layers-2)):
        for j in range(Layer_dim[l+1]):
            Deltas[l][j]=0
            for k in range(Layer_dim[l+2]):
                Deltas[l][j]+=Weights[l+1][j][k]*Deltas[l+1][k]*hidden_func(Signals[l][j],True)
                    # I use [Signals[l][j]] in hidden_func because I have assumed that the input is a list.
                    #the [0] cancels the [...] in the input!
                    

In [3]:
#read data
X=[] #data to fit
VX=[] #data to check
temp1=[1]*5

temp1.append(1)
for line in open('data.txt'):
    
    temp=map(float,line.rstrip('\n').split(','))
    
    if np.random.choice(temp1)==1:
        X.append(temp)
    else:
        VX.append(temp)
    
#normalize each column to be in (0,1]
max_col=np.max(X,axis=0)
if len(VX)!=0:
    max_colV=np.max(VX,axis=0)
else:
    max_colV=np.max(X,axis=0)
#min_col=np.min(X,axis=0)
#min_colV=np.min(VX,axis=0)
for i in range(len(X)):
    for j in range(len(X[i])):
        X[i][j]=(X[i][j])/max([max_col[j],max_colV[j]])
    X[i].insert(len(X[i])-1,1)#add bias
for i in range(len(VX)):
    for j in range(len(VX[i])):
        VX[i][j]=(VX[i][j])/max([max_col[j],max_colV[j]])
    VX[i].insert(len(VX[i])-1,1)#add bias. I don't need it because I don't use it for training.
    
X

[[1.0, 1.0, 1, 1.0],
 [1.0, 0.0, 1, 1.0],
 [0.0, 1.0, 1, 1.0],
 [0.0, 0.0, 1, 0.0]]

In [4]:
datapoints=len(X)
d=[len(X[0])-2,2,1] #dimension of each layer ( initial, [hidden] , 1 output)
Learning_Rate=1
Runs=100000
L=len(d) #number of layers
#--------------------------------------------------------------------------------------------------
#initialize weights
np.random.seed(1)
w=[]
for l in range(0,L-1):
    temp1=[]
    for i in range(d[l]+1):
        temp1.append([])
        for j in range(0,d[l+1]):
            temp1[i].append(np.random.choice([np.random.random()*2-1]))
    w.append(temp1)

#--------------------------------------------------------------------------------------------------
#first, for simplicity, define the x's (empty array)
xl=[]
for l in range(L-1):
    xl.append([0]*(d[l]+1)) #I've added d[l]+1 for the bias
#define the last xl outside the loop, because it has no bias
xl.append([0])    
#add the bias for each l
for l in range(L-1):
    xl[l][len(xl[l])-1]=1    
#I have to calculate the signals's, so in the same manner I can initialize the signals
signal=[]
for l in range(L-1):
    signal.append([0]*(d[l+1])) #it's d[l+1] because there are as much signals in l as x's in l+1 (excluding the bias)
#Since I have as many deltas as signals, I set delta=signal
delta=signal[:]    

#--------------------------------------------------------------------------------------------------
delta0=0
conv=0
Error=0
st=0
stt=0
for epoch in range(1,Runs):
    #calculate x[l] for one data point (and the signals!)
    n=np.random.randint(len(X))
    lenX=len(X[n])
    xl[0]=X[n][:lenX-1]
    xl[0]=np.array(xl[0])    
    X_calculation(signal,xl,w,L )
    BackProp(signal,w,error_func(xl[L-1][0],X[n][lenX-1:lenX][0],True),delta,L,d)
    W_update(xl,delta,L,d,Learning_Rate,w)
                
#-----------------------------------------------------------------------------------------------------------                

#-----------------------------------------------------------------------------------------------------------                
    
    if st<datapoints*1000:  
        st+=1
        if X[n][lenX-1]!=0:
            delta0+=np.abs((xl[L-1][0]-X[n][lenX-1])/float(X[n][lenX-1]))
        if X[n][lenX-1]==0:
            delta0+=np.abs(xl[L-1][0]-X[n][lenX-1])/10
        if stt==2:
            Error+=error_func(xl[L-1][0],X[n][lenX-1:lenX][0])
    else:
        stt+=1
    if stt==1:
        average1=delta0/float(st)
        delta0=0
        stt+=1
        st=0
    if stt==3:
        average2=delta0/float(st)       
        print'epochs: '+str(epoch)+'\t\t|\t\t'+ str("{:.5f}".format(average1))+"\t\t"+str("{:.5f}".format(average2))
       
        #------------calculate the delta's for the average error (it's slower but sometimes it converges quicker)! 
        #X_calculation(signal,xl,w,L )
        #BackProp(signal,w,Error,delta,L,d)
        #W_update(xl,delta,L,d,Learning_Rate,w)

#------------------------------------------------------------------------------------------------     
        delta0=0
        Error=0
        st=0
        stt=0            
#------------------------------------------------------------------------------------------------            
        #play around with the following if to change the point of exitting the loop

        if average1==0 and average2<=0.05:
            #print 'break epoch' '\t' + str(epoch)
            conv=1
            break
    
        if np.abs(average1)<0.01 and np.abs(average1-average2)/np.abs(average1)<=0.05:
            #print 'break epoch' '\t' + str(epoch)
            conv=1
            break
        if np.abs(average1)<0.30 and np.abs(average1-average2)/np.abs(average1)<0.1 and Learning_Rate>0.00001:
            Learning_Rate=Learning_Rate/2
            #print 'Learning Rate: \t '+ str(Learning_Rate)
        if Learning_Rate<=0.00001:
            Learning_Rate=Learning_Rate*20
            #print 'Learning Rate: \t '+ str(Learning_Rate)
if conv==1:
    print 'We have convergence'
else:
    print 'We don\'t really have convergence'
print 'model:'
print w

epochs: 8002		|		0.02490		0.00167
epochs: 16004		|		0.00008		0.00000
epochs: 24006		|		0.00000		0.00000
epochs: 32008		|		0.00000		0.00000
epochs: 40010		|		0.00000		0.00000
epochs: 48012		|		0.00000		0.00000
epochs: 56014		|		0.00000		0.00000
We have convergence
model:
[[[-3.225265275180998, -0.465514875024822], [-3.8519100096264927, -0.7747048498084319], [-0.462269072919738, -1.1396011314087777]], [[-2.9387439689785224], [0.8505728570209818], [0.9294973560514533]]]


In [5]:
test=1
if test==1:
    print '\tFit' + '\t\t'+'Target'
    for n in range(int(len(X))):
        xl[0]=X[n][:len(X[n])-1]
        xl[0]=np.array(xl[0])
        X_calculation(signal,xl,w,L)    
        print str("{:>10.2}".format(xl[L-1][0]))+ ' \t '+ str("{:>10.3}".format(X[n][len(X[n])-1])) 
    for n in range(int(len(VX))):
        xl[0]=VX[n][:len(VX[n])-1]
        xl[0]=np.array(xl[0])
        X_calculation(signal,xl,w,L)    
        print str("{:>10.2}".format(xl[L-1][0]))+ ' \t '+ str("{:>10.3}".format(VX[n][len(VX[n])-1])) 
        


	Fit		Target
       1.0 	        1.0
       1.0 	        1.0
       1.0 	        1.0
       0.0 	        0.0
